In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd


df_postings = pd.read_csv("C:/Users/marioreyes/Desktop/Data Analysis Practice/Python/Project/Job Postings/Uncleaned_DS_jobs.csv")
df_postings.drop(columns=['index','Competitors'], inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          672 non-null    object 
 1   Salary Estimate    672 non-null    object 
 2   Job Description    672 non-null    object 
 3   Rating             672 non-null    float64
 4   Company Name       672 non-null    object 
 5   Location           672 non-null    object 
 6   Headquarters       672 non-null    object 
 7   Size               672 non-null    object 
 8   Founded            672 non-null    int64  
 9   Type of ownership  672 non-null    object 
 10  Industry           672 non-null    object 
 11  Sector             672 non-null    object 
 12  Revenue            672 non-null    object 
dtypes: float64(1), int64(1), object(11)
memory usage: 68.4+ KB


In [ ]:
pd.set_option('display.max_columns', None)

           Job Title               Salary Estimate  \
0  Sr Data Scientist  $137K-$171K (Glassdoor est.)   
1     Data Scientist  $137K-$171K (Glassdoor est.)   
2     Data Scientist  $137K-$171K (Glassdoor est.)   
3     Data Scientist  $137K-$171K (Glassdoor est.)   
4     Data Scientist  $137K-$171K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Description\n\nThe Senior Data Scientist is re...     3.1   
1  Secure our Nation, Ignite your Future\n\nJoin ...     4.2   
2  Overview\n\n\nAnalysis Group is one of the lar...     3.8   
3  JOB DESCRIPTION:\n\nDo you have a passion for ...     3.5   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

              Company Name       Location            Headquarters  \
0         Healthfirst\n3.1   New York, NY            New York, NY   
1             ManTech\n4.2  Chantilly, VA             Herndon, VA   
2      Analysis Group\n3.8     Boston, MA              Boston, MA   
3             

In [ ]:
#Data Cleaning and Formatting
#print(df_postings.groupby('Salary Estimate')['Salary Estimate'].count())

#Splitting Salary Estimate Column
#Splitting salary into two columns (since we know there is only one - in each entry)
df_postings[['Lower Salary Estimate','new']]= df_postings['Salary Estimate'].str.split('-',expand=True)
df_postings['Lower Salary Estimate'] = df_postings['Lower Salary Estimate'].str.strip('$K').astype('int')*1000

#Splitting salary into two columns. Here we use n=1 since there are multiple " " in each entry
df_postings[['High Salary Estimate','new_2']]= df_postings['new'].str.split('K',n=1,expand=True)
df_postings['High Salary Estimate'] = df_postings['High Salary Estimate'].str.strip('$K ').astype('int')*1000
#Dropping Columns that we generated and no longer need
df_postings.drop(columns=['new','new_2'], inplace=True)


#Cleaning Company Name
df_postings[['Company Name2','new']]= df_postings['Company Name'].str.split('\n',n=1,expand=True)
df_postings.drop(columns=['new','Company Name'], inplace=True)

df_postings.rename(columns={'Company Name2': 'Company Name'}, inplace=True)
#There were some company names that did not have the decimal points. Checking one of those instances to make sure the strip worked correctly
#print(df_postings['Company Name'].str.contains('Radical Convergence').any())




In [ ]:
#Splitting the Size column

df_postings[['Size2','new']]= df_postings['Size'].str.split(' ',n=1,expand=True)
#Looks like there are some values with "+" so lets strip those
df_postings['Size2'] = df_postings['Size2'].str.strip('+')
#Looks like there are some values labeled as unknown. Let's change those to 1
df_postings['Size2'] = df_postings['Size2'].replace('Unknown', '1')
#There's also values that are -1. Looks like those values have a bunch of missing entries. Let's drop them
df_postings = df_postings[df_postings['Size2'] != "-1"]

df_postings['Size2'] = df_postings['Size2'].astype('int')
df_postings.rename(columns={'Size2': 'Lower Staff Size'}, inplace=True)


df_postings[['Size3','new2']]= df_postings['new'].str.split('to ',n=2,expand=True)
df_postings[['Size4','new3']]= df_postings['new2'].str.split(' ',n=1,expand=True)
#Seems like we can't convert column into int bc there are None Values. Let's check the breakdown of the column
df_postings.rename(columns={'Size4': 'Higher Staff Size'}, inplace=True)
df_postings.drop(columns=['new','new2','new3','Size3'], inplace=True)


In [ ]:
df_postings.info()
#Looks like there are None values in the higher staff column. Let's see how those values look like compared to low staff colimn

<class 'pandas.core.frame.DataFrame'>
Index: 645 entries, 0 to 671
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Job Title              645 non-null    object 
 1   Salary Estimate        645 non-null    object 
 2   Job Description        645 non-null    object 
 3   Rating                 645 non-null    float64
 4   Location               645 non-null    object 
 5   Headquarters           645 non-null    object 
 6   Size                   645 non-null    object 
 7   Founded                645 non-null    int64  
 8   Type of ownership      645 non-null    object 
 9   Industry               645 non-null    object 
 10  Sector                 645 non-null    object 
 11  Revenue                645 non-null    object 
 12  Lower Salary Estimate  645 non-null    int64  
 13  High Salary Estimate   645 non-null    int64  
 14  Company Name           645 non-null    object 
 15  Lower Staff

In [ ]:
print(df_postings[df_postings['Higher Staff Size'].isnull()][['Lower Staff Size','Higher Staff Size']])
print(df_postings.groupby('Higher Staff Size', dropna=False)['Higher Staff Size'].count())

#The original dataset had entries that read "10000+". Since we dont have a max value, for this practice we will create a max value of 20000
df_postings.loc[df_postings['Lower Staff Size'] == 10000, 'Higher Staff Size'] = df_postings.loc[df_postings['Lower Staff Size'] == 10000, 'Higher Staff Size'].fillna('20000')

#Looks like there is another None value when the lower staff size is 1. If you remember we changed those with "unknown"
#values to 1. We don't have a range of values for these companies, so we will set a max value of 50
df_postings.loc[df_postings['Lower Staff Size'] == 1, 'Higher Staff Size'] = df_postings.loc[df_postings['Lower Staff Size'] == 1, 'Higher Staff Size'].fillna(50)

#There are no more None values in the column. So now let's convert the column into an int
df_postings['Higher Staff Size'] = df_postings['Higher Staff Size'].astype('int')


     Lower Staff Size Higher Staff Size
6               10000              None
31              10000              None
41              10000              None
45              10000              None
51              10000              None
..                ...               ...
644             10000              None
645             10000              None
651             10000              None
659             10000              None
661             10000              None

[97 rows x 2 columns]
Higher Staff Size
1000      77
10000     61
200      135
50        86
500       85
5000     104
NaN        0
Name: Higher Staff Size, dtype: int64


In [ ]:
print(df_postings.info())

<class 'pandas.core.frame.DataFrame'>
Index: 645 entries, 0 to 671
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Job Title              645 non-null    object 
 1   Salary Estimate        645 non-null    object 
 2   Job Description        645 non-null    object 
 3   Rating                 645 non-null    float64
 4   Location               645 non-null    object 
 5   Headquarters           645 non-null    object 
 6   Size                   645 non-null    object 
 7   Founded                645 non-null    int64  
 8   Type of ownership      645 non-null    object 
 9   Industry               645 non-null    object 
 10  Sector                 645 non-null    object 
 11  Revenue                645 non-null    object 
 12  Lower Salary Estimate  645 non-null    int64  
 13  High Salary Estimate   645 non-null    int64  
 14  Company Name           645 non-null    object 
 15  Lower Staff

In [ ]:
print(df_postings['Revenue'])

0        Unknown / Non-Applicable
1          $1 to $2 billion (USD)
2      $100 to $500 million (USD)
3      $100 to $500 million (USD)
4        Unknown / Non-Applicable
                  ...            
665     $50 to $100 million (USD)
666    $100 to $500 million (USD)
667      Unknown / Non-Applicable
670        $1 to $5 million (USD)
671        $1 to $2 billion (USD)
Name: Revenue, Length: 645, dtype: object


In [ ]:
df_postings[['Revenue2','new']]= df_postings['Revenue'].str.split(' ',n=1,expand=True)
df_postings[['Revenue3','new2']]= df_postings['new'].str.split('$',expand=True)
df_postings[['Revenue4','new3']]= df_postings['new2'].str.split(' ',n=1,expand=True)
df_postings[['Revenue5','new4']]= df_postings['new3'].str.split(' ',n=1,expand=True)


#Creating lower bond for Revenue
df_postings['Revenue2'] = df_postings['Revenue2'].str.strip('$+')
df_postings['Revenue2'] = df_postings['Revenue2'].replace('Unknown', '0')
#There is a string that is Less than 1Million so we will make the lower level 0
df_postings['Revenue2'] = df_postings['Revenue2'].replace('Less', '0')
df_postings['Revenue2'] = df_postings['Revenue2'].astype('int')
df_postings.rename(columns={'Revenue2': 'Lower Revenue'}, inplace=True)

#There's an instance where the revenue is " $10+ billion (USD) " since there is no upper bound we will just set it as 20 billion
df_postings.loc[df_postings['Revenue'] == '$10+ billion (USD)', 'Revenue4'] = df_postings.loc[df_postings['Revenue'] == '$10+ billion (USD)', 'Revenue4'].fillna('20')


#Creating higher bond for revenue
df_postings['Revenue4'] = df_postings['Revenue4'].fillna(0)
df_postings['Revenue4'] = df_postings['Revenue4'].astype('int')
df_postings.rename(columns={'Revenue4': 'Higher Revenue'}, inplace=True)


#Converting values to actual numbers. We have to be careful since the values could be billion or million
df_postings.loc[df_postings['Revenue5'] == 'billion', 'Lower Revenue'] *= 1000000000
df_postings.loc[df_postings['Revenue5'] == 'billion', 'Higher Revenue'] *= 1000000000
df_postings.loc[df_postings['Revenue5'] == 'million', 'Lower Revenue'] *= 1000000
df_postings.loc[df_postings['Revenue5'] == 'million', 'Higher Revenue'] *= 1000000

In [ ]:
print(df_postings.groupby('Higher Revenue')['Higher Revenue'].count())
#Looks like there are still some values that are not being converted

Higher Revenue
0              213
20              63
1000000         14
5000000         31
10000000        14
25000000        41
50000000        36
100000000       31
500000000       94
1000000000      19
2000000000      36
5000000000      45
10000000000      8
Name: Higher Revenue, dtype: int64


In [ ]:
print(df_postings[['Revenue','Revenue5','Lower Revenue','Higher Revenue']][80:100])
#From this view we see that we have an entry "$10+ billion (USD)" where the split did not work properly. So these values are not being converted to billions

                       Revenue Revenue5  Lower Revenue  Higher Revenue
80          $10+ billion (USD)     None             10              20
81    $25 to $50 million (USD)  million       25000000        50000000
82    $25 to $50 million (USD)  million       25000000        50000000
83    Unknown / Non-Applicable     None              0               0
84    $25 to $50 million (USD)  million       25000000        50000000
85    Unknown / Non-Applicable     None              0               0
86    Unknown / Non-Applicable     None              0               0
87    $10 to $25 million (USD)  million       10000000        25000000
88    Unknown / Non-Applicable     None              0               0
89          $10+ billion (USD)     None             10              20
90          $10+ billion (USD)     None             10              20
91     $5 to $10 million (USD)  million        5000000        10000000
92      $1 to $2 billion (USD)  billion     1000000000      2000000000
93    

In [ ]:
#Here we will just lookup where those entries are and multiply its values
df_postings.loc[df_postings['Revenue'] == '$10+ billion (USD)', 'Lower Revenue'] *= 1000000000
df_postings.loc[df_postings['Revenue'] == '$10+ billion (USD)', 'Higher Revenue'] *= 1000000000

#To keep it consistent we will also update the Revenue5 column
df_postings.loc[df_postings['Revenue'] == '$10+ billion (USD)', 'Revenue5'] = 'billion'

df_postings.rename(columns={'Revenue5': 'Currency Multiplier'}, inplace=True)

#print(df_postings[['Revenue','Currency Multiplier','Lower Revenue','Higher Revenue']][80:100])

#After further expection, there is a revenue titled "$500 million to $1 billion (USD)" which is being labeled as "billion". This is true for the high revenue values but not the low ones.
#Let's fix this by hard coding the low revenue
df_postings.loc[df_postings['Revenue'] == "$500 million to $1 billion (USD)", 'Lower Revenue'] = df_postings.loc[df_postings['Revenue'] == "$500 million to $1 billion (USD)", 'Lower Revenue'] = int(500000000)

#Double checking with an example
print(df_postings[df_postings['Type of ownership']=="Government"][['Revenue','Lower Revenue','Higher Revenue','Currency Multiplier']])

                              Revenue  Lower Revenue  Higher Revenue  \
12   $500 million to $1 billion (USD)      500000000      1000000000   
48           Unknown / Non-Applicable              0               0   
177  $500 million to $1 billion (USD)      500000000      1000000000   
264  $500 million to $1 billion (USD)      500000000      1000000000   
275            $1 to $2 billion (USD)     1000000000      2000000000   
300                $10+ billion (USD)    10000000000     20000000000   
323          Unknown / Non-Applicable              0               0   
515          Unknown / Non-Applicable              0               0   
589            $2 to $5 billion (USD)     2000000000      5000000000   
595          Unknown / Non-Applicable              0               0   

    Currency Multiplier  
12              billion  
48                 None  
177             billion  
264             billion  
275             billion  
300             billion  
323                None  

In [ ]:
df_postings.info()
df_postings.drop(columns=['new','new2','Revenue3','new4','new3'], inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 645 entries, 0 to 671
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Job Title              645 non-null    object 
 1   Salary Estimate        645 non-null    object 
 2   Job Description        645 non-null    object 
 3   Rating                 645 non-null    float64
 4   Location               645 non-null    object 
 5   Headquarters           645 non-null    object 
 6   Size                   645 non-null    object 
 7   Founded                645 non-null    int64  
 8   Type of ownership      645 non-null    object 
 9   Industry               645 non-null    object 
 10  Sector                 645 non-null    object 
 11  Revenue                645 non-null    object 
 12  Lower Salary Estimate  645 non-null    int64  
 13  High Salary Estimate   645 non-null    int64  
 14  Company Name           645 non-null    object 
 15  Lower Staff

In [ ]:
df_postings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 645 entries, 0 to 671
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Job Title              645 non-null    object 
 1   Salary Estimate        645 non-null    object 
 2   Job Description        645 non-null    object 
 3   Rating                 645 non-null    float64
 4   Location               645 non-null    object 
 5   Headquarters           645 non-null    object 
 6   Size                   645 non-null    object 
 7   Founded                645 non-null    int64  
 8   Type of ownership      645 non-null    object 
 9   Industry               645 non-null    object 
 10  Sector                 645 non-null    object 
 11  Revenue                645 non-null    object 
 12  Lower Salary Estimate  645 non-null    int64  
 13  High Salary Estimate   645 non-null    int64  
 14  Company Name           645 non-null    object 
 15  Lower Staff

In [ ]:
#Data Analysis 1
#To make a sorted hisplot we need to sort the dataframe. For this case we will create a new df to plot ownership counts
ownership_counts = df_postings['Type of ownership'].value_counts()
ownership_counts_sorted = ownership_counts.sort_values(ascending=False)

#The above code gives us a Series. We need a dataframe to make the hisplot work
ownership_df = pd.DataFrame({'ownership': ownership_counts_sorted.index, 'count': ownership_counts_sorted.values})

ax1 = sns.histplot(
    data=ownership_df,
    x="ownership",
    weights='count',
    discrete=True)
plt.xticks(rotation=45, horizontalalignment='right')
ax1.bar_label(ax1.containers[0],fontsize=7 ) #adding labels to bar chart
plt.show()

#We could make it simpler by using a barplot instead. This can make a Series, so no need for a df convertion
#ax2 = sns.barplot(data=ownership_df,
#            x='ownership',
#            y='count')
#plt.xticks(rotation=45, horizontalalignment='right')
#ax2.bar_label(ax2.containers[0],fontsize=7 ) #adding labels to bar chart
#plt.show()

                         ownership  count
0                Company - Private    397
1                 Company - Public    153
2           Nonprofit Organization     36
3   Subsidiary or Business Segment     28
4                       Government     10
5               Other Organization      5
6          Private Practice / Firm      4
7                          Unknown      4
8             College / University      3
9                    Self-employed      2
10                        Contract      2
11                        Hospital      1


In [ ]:
#Data Analysis 1 Summary
#Most of the ownerships are of private company. But what does that tell us about revenue and salary?

In [ ]:
#Data Analysis 2. Let's see the staff and revenue numbers of each of the ownerships

#We have two columns for lower and high salary estimate values. Let's make them into one column, and have a new column to determine the category "high or low" estimate
#Think of it as Unpivoting from PowerBI. This will allow us to plot everything in one visual
df_melted = df_postings.melt(id_vars='Type of ownership', value_vars=['Lower Salary Estimate', 'High Salary Estimate'], var_name='value_type', value_name='value')
plt.figure(figsize=(9,6))
sns.boxplot(data=df_melted, x='Type of ownership', y='value', hue='value_type', 
            #Let's not show the outliners so we can see the visual better
            showfliers=False)
plt.xticks(rotation=45, horizontalalignment='right')
plt.title('Salary Estimate by Ownership')
plt.show()

#Let's do this for Revenue
df_melted2 = df_postings.melt(id_vars='Type of ownership', value_vars=['Lower Revenue', 'Higher Revenue'], var_name='value_type', value_name='value')
plt.figure(figsize=(9,6))
ax3 = sns.boxplot(data=df_melted2, x='Type of ownership', y='value', hue='value_type', 
            #Let's not show the outliners so we can see the visual better
            showfliers=False)
plt.xticks(rotation=45, horizontalalignment='right')
plt.title('Revenue Estimate by Ownership W/ Company - Public Included')
plt.show()

#Looks like the revenue for Company - Public is extremely high so it cuts of the other categories. Let's do another visual but this time lets take out Company - Public
df_melted3 = df_melted2
df_melted3 = df_melted3[df_melted3['Type of ownership'] != 'Company - Public']
ax4 = sns.boxplot(data=df_melted3, x='Type of ownership', y='value', hue='value_type', 
            #Let's not show the outliners so we can see the visual better
            showfliers=False)
plt.xticks(rotation=45, horizontalalignment='right')
plt.title('Revenue Estimate by Ownership W/O Company - Public')
plt.show()




            Type of ownership      value_type       value
0      Nonprofit Organization   Lower Revenue           0
1            Company - Public   Lower Revenue  1000000000
2     Private Practice / Firm   Lower Revenue   100000000
3            Company - Public   Lower Revenue   100000000
4           Company - Private   Lower Revenue           0
...                       ...             ...         ...
1285        Company - Private  Higher Revenue   100000000
1286         Company - Public  Higher Revenue   500000000
1287        Company - Private  Higher Revenue           0
1288        Company - Private  Higher Revenue     5000000
1289         Company - Public  Higher Revenue  2000000000

[1290 rows x 3 columns]
            Type of ownership      value_type      value
0      Nonprofit Organization   Lower Revenue          0
2     Private Practice / Firm   Lower Revenue  100000000
4           Company - Private   Lower Revenue          0
5           Company - Private   Lower Revenue      

In [ ]:
#Data Analysis 2 Summary.
#Althought most companies are Private, their low salary estimates are not as competitive as other types (non profits, public, government, other)
#Their high salary estimates do a better job competing with other ownership types but still most of the job listings are less competitive than the above types

#For Revenue
#Private companies rank 7th in their revenue (for both low and high revenue categories) compared to the other ownership types

In [ ]:
#Data Analysis 3
#Let's graph the number of postings for each location

sns.histplot(
    data=df_postings,
    x="Location"
)
plt.xticks(rotation=45, horizontalalignment='right')
plt.title('Number of locations')
plt.show()

#Looks like there is too many locations. Let's just show the top 20
df_location = df_postings["Location"].value_counts()
df_location = df_location[:20].sort_values(ascending=False)
#We could convert the Serie above to a dataframe so seaborn can plot it. But we can also just specifically tell seaborn to plot using index and values
#df_location = pd.DataFrame({"Location": df_location.index, 'count': df_location.values})

#Plotting Series, and not a dataframe
ax5 = sns.barplot(
    x=df_location.index,
    y=df_location.values
)
plt.xticks(rotation=45, horizontalalignment='right')
ax5.bar_label(ax5.containers[0],fontsize=7 ) #adding labels to bar chart
plt.title('Top 20 locations')
plt.show()


#Let's investigate the salary estimates for the top 20 locations with job listings
df_melted4 = df_postings.melt(id_vars='Location', value_vars=['Lower Salary Estimate', 'High Salary Estimate'], var_name='value_type', value_name='value')
#Here we have all of the locations included. Let just include the top 20
#First we need the list of the top 20 locations
df_location = pd.DataFrame({"Location": df_location.index, 'count': df_location.values})
df_20 = df_location.iloc[:, 0].tolist()

#Now we filter df_melted4 to only include the locations from the df_20 list
df_melted5 = df_melted4[df_melted4['Location'].isin(df_20)]



ax6 = sns.boxplot(data=df_melted5, 
                  x='Location', 
                  y='value', 
                  hue='value_type', 
                  order=df_20, #Not necessary but this manually sorts the x axis in the order of the df_20 list 
            #Let's not show the outliners so we can see the visual better
            showfliers=False)
plt.xticks(rotation=45, horizontalalignment='right')
plt.title('Revenue Estimate by Job Posting Location')
plt.show()

['San Francisco, CA', 'New York, NY', 'Washington, DC', 'Boston, MA', 'Chicago, IL', 'Herndon, VA', 'Cambridge, MA', 'McLean, VA', 'United States', 'Chantilly, VA', 'Santa Clara, CA', 'Saint Louis, MO', 'Atlanta, GA', 'Gaithersburg, MD', 'San Diego, CA', 'Reston, VA', 'Redwood City, CA', 'Remote', 'Cincinnati, OH']


In [ ]:

#Data Analysis 3 Summary
#There are many locations in the job postings, but SF, NY, and Washington DC have the most postings
#Although SF has the most posting, they don't have a competitive lower salary estimate compared to the next ~6 locations with the most job postings
#This is not entirely true for their high salary estimate, which is competitive to many locations, but falls short to NY, Chicago, MCLean and Reston, VAS

In [ ]:
#Data Analysis 4 Correlation
#Plotting all numeric columns in a correlation matrix
correlation_matrix = df_postings.corr(numeric_only=True, method="pearson")
plt.figure(figsize=(14,14), dpi = 40)
sns.heatmap(correlation_matrix, annot=True)

plt.title("Correlation Metrix for Numeric Features")

plt.yticks(rotation=0)
plt.show()

In [ ]:
#Lets find the correlation again but this time lets include the non number columns
df_numerized = df_postings.copy()

for col_name in df_numerized.columns:
    if(df_numerized[col_name].dtype == "object"):
        df_numerized[col_name] = df_numerized[col_name].astype("category") 
        df_numerized[col_name] = df_numerized[col_name].cat.codes

correlation_matrix = df_numerized.corr(numeric_only=True, method="pearson")


plt.figure(figsize=(20,14), dpi = 40)
sns.heatmap(correlation_matrix, annot=True)

plt.title("Correlation Metrix for All Columns")

plt.yticks(rotation=0)
plt.show()

In [ ]:
#Matrix is big. Let's convert into a tableu and show the top correlated pairs
correlation_mat = df_numerized.corr(numeric_only=True)
corr_pairs = correlation_mat.unstack()
sorted_pairs = corr_pairs.sort_values(ascending=False)
#print(sorted_pairs[((sorted_pairs) >0.5) & (sorted_pairs<1)])
print(sorted_pairs[((sorted_pairs<0))])

Salary Estimate        Location                -0.000602
Location               Salary Estimate         -0.000602
Lower Staff Size       High Salary Estimate    -0.001235
High Salary Estimate   Lower Staff Size        -0.001235
Lower Salary Estimate  Higher Staff Size       -0.001313
                                                  ...   
Salary Estimate        Lower Salary Estimate   -0.394729
Higher Staff Size      Size                    -0.395399
Size                   Higher Staff Size       -0.395399
Revenue                Currency Multiplier     -0.694516
Currency Multiplier    Revenue                 -0.694516
Length: 208, dtype: float64


In [ ]:
#Data Analysis #4 Summary
#There is a strong correlation between low and high revenue
#There is a strong correlation between low and high staff size
#There is a strong correlation between low and high salary estimate
#There is a week correlation between location and salary estimate
#There is a week correlation between founded data and salary estimate

In [ ]:
#Data Analysis 5
#Let's see the median salary for job titles
#Checking how many job titles there are.
print(df_postings['Job Title'].value_counts())

#There are multiple different job titles. Let's create a function where it shortens these job postings
def title_short(title):
    if "manager" in title.lower():
        return "data manager"
    if "director" in title.lower() or "president" in title.lower():
        return "data director or president"
    if 'scientist' in title.lower():
        return "data scientist"
    if "analyst" in title.lower():
        return "data analyst"
    if "machine learning" in title.lower():
        return "machine learning"
    if "science" in title.lower():
        return "data scientist"
    if "engineer" in title.lower():
        return "data engineer"
    if "architect" in title.lower() or "architecture" in title.lower():
        return "data architect"
    if "modeler" in title.lower():
        return "data modeler"
    else:
        return "other"

df_postings["Job Title Short"] = df_postings['Job Title'].apply(title_short)


Job Title
Data Scientist                                                      314
Data Engineer                                                        26
Senior Data Scientist                                                19
Machine Learning Engineer                                            15
Data Analyst                                                         12
                                                                   ... 
Sr. Research Associate/ Scientist, NGS prep & Molecular Genomics      1
Developer III - Data Science                                          1
Hydrogen/Tritium Materials Scientist (Experienced)                    1
Data Scientist/Data Analytics Practitioner                            1
AI/ML - Machine Learning Scientist, Siri Understanding                1
Name: count, Length: 170, dtype: int64


In [ ]:
#df_postings.info()
print(df_postings['Job Title Short'].value_counts())

#Creating a groupby statement to find the median lower and high salary for each job title
print(df_postings.groupby('Job Title Short')[['Lower Salary Estimate','High Salary Estimate']].median().sort_values(ascending=False, by='Lower Salary Estimate'))

#Let's say we want to group both salary estimates into one. Let's create a new column to get the average for each job title

df_postings['Average Salary'] = (df_postings['Lower Salary Estimate'] + df_postings['High Salary Estimate'])/2
print(df_postings.groupby('Job Title Short')['Average Salary'].median().sort_values(ascending=False))


Traceback (most recent call last):
  File "C:\Users\marioreyes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Job Title Short'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\marioreyes\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callab

In [ ]:
#Data Summary 5 Summary
#Data architect jobs tend to have the highest pay BUT this assumption is made up from just 1 available job posting
#The second highest pay job is data director or president, but its made up of only 4 job listings. Making it also unreliable
#Data scientiest roles, which has over 500+ job postings, rank 3rd in salary. Althought they aren't the highest compensated position, this figure does give us a better representation of what someone should expect from this job title


                                     Job Title              Salary Estimate  \
80          Real World Science, Data Scientist  $79K-$131K (Glassdoor est.)   
81                              Data Scientist  $79K-$131K (Glassdoor est.)   
82                               Data Engineer  $79K-$131K (Glassdoor est.)   
83  Data Scientist - Image and Video Analytics  $79K-$131K (Glassdoor est.)   

                                      Job Description  Rating  \
80  Title: Real World Science, Data Scientist\nLoc...     4.0   
81  The Senior Data Scientist will build and impro...     3.6   
82  Job Description\n\nWhy consider OPI, and why d...     4.7   
83  Company Industry: Technology\nOpportunity: The...     4.9   

            Location               Headquarters                 Size  Founded  \
80  Gaithersburg, MD  Cambridge, United Kingdom     10000+ employees     1913   
81    Washington, DC              Rockville, MD  51 to 200 employees     2001   
82   Saint Louis, MO            Min

In [ ]:
#Data Analysis 6
#We just discovered that some jobs are either entry, junior, or senior positions.
#Let's find the median salary for these positions and compare them

#Making a function where it will label a job as junior/entry or senior

def senority(Job_Title):
    if "junior" in Job_Title.lower() or "entry" in Job_Title.lower() or "jr" in Job_Title.lower():
        return "Entry"
    if "senior" in Job_Title.lower() or "sr" in Job_Title.lower() or "experienced" in Job_Title.lower():
        return "Senior"
    else:
        return "Not Labeled"
    
df_postings['Seniority'] = df_postings['Job Title'].apply(senority)
print(df_postings['Seniority'].value_counts())
print(df_postings.groupby(['Job Title Short','Seniority'])['Average Salary'].median())

#We could also use a pivot table to show values
#pd.pivot_table(df_postings, index=['Job Title Short','Seniority'], values= 'Average Salary', aggfunc='median')

Seniority
Not Labeled    564
Senior          79
Entry            2
Name: count, dtype: int64
Traceback (most recent call last):
  File "c:\Users\marioreyes\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 17, in <module>
  File "C:\Users\marioreyes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\frame.py", line 9183, in groupby
    return DataFrameGroupBy(
        obj=self,
    ...<7 lines>...
        dropna=dropna,
    )
  File "C:\Users\marioreyes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\groupby\groupby.py", line 1329, in __init__
    grouper, exclusions, obj = get_grouper(
                               ~~~~~~~~~~~^
        obj,
        ^^^^
    ...<5 lines>...
    

In [ ]:
pd.pivot_table(df_postings, index=['Rating'], values= 'Average Salary', aggfunc='median')
#Checking if the rating of the company has any affect to salary
#From the correlation earlier, we know that rating and lower/high salary estimates don't have a large correlation
#This table is giving us a visual on how the median salary is around 100-120K regardless of rating

Traceback (most recent call last):
  File "c:\Users\marioreyes\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
  File "C:\Users\marioreyes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\reshape\pivot.py", line 102, in pivot_table
    table = __internal_pivot_table(
        data,
    ...<9 lines>...
        sort,
    )
  File "C:\Users\marioreyes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\reshape\pivot.py", line 148, in __internal_pivot_table
    raise KeyError(i)
KeyError: 'Average Salary'



In [ ]:
#Line of best fit for salary
# Create subset of data
ols_data = df_postings[["High Salary Estimate","Lower Salary Estimate"]]

# Rename columns to remove spaces
ols_data.rename(columns={'High Salary Estimate': 'High_Salary_Estimate'}, inplace=True)
ols_data.rename(columns={'Lower Salary Estimate': 'Lower_Salary_Estimate'}, inplace=True)

#Write formula as text. Dependent variable (y) first, then independent (x)
ols_formula = "High_Salary_Estimate ~ Lower_Salary_Estimate"

#Import libraries
from statsmodels.formula.api import ols

#build OLS
model = ols(formula=ols_formula, data=ols_data).fit()

print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     High_Salary_Estimate   R-squared:                       0.822
Model:                              OLS   Adj. R-squared:                  0.822
Method:                   Least Squares   F-statistic:                     2977.
Date:                  Fri, 28 Mar 2025   Prob (F-statistic):          1.74e-243
Time:                          13:06:24   Log-Likelihood:                -7313.7
No. Observations:                   645   AIC:                         1.463e+04
Df Residuals:                       643   BIC:                         1.464e+04
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept 

In [ ]:
sns.regplot(x = 'Lower_Salary_Estimate', y = "High_Salary_Estimate", data = ols_data)
plt.show()

In [ ]:
# Create subset of data
#print(df_postings.head(1))
ols_data2 = df_postings[["High Salary Estimate","Founded"]]
ols_data2 = ols_data2[ols_data2['Founded'] != -1]
#print(ols_data2)


# Rename columns to remove spaces
ols_data2.rename(columns={'High Salary Estimate': 'High_Salary_Estimate'}, inplace=True)

#Write formula as text. Dependent variable (y) first, then independent (x)
ols_formula = "High_Salary_Estimate ~ Founded"

#Import libraries
from statsmodels.formula.api import ols

#build OLS
model = ols(formula=ols_formula, data=ols_data2).fit()

print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     High_Salary_Estimate   R-squared:                       0.000
Model:                              OLS   Adj. R-squared:                 -0.002
Method:                   Least Squares   F-statistic:                   0.01189
Date:                  Fri, 28 Mar 2025   Prob (F-statistic):              0.913
Time:                          12:59:27   Log-Likelihood:                -6731.5
No. Observations:                   554   AIC:                         1.347e+04
Df Residuals:                       552   BIC:                         1.348e+04
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.556e+05   9.52e+04

In [ ]:
sns.regplot(x = 'Founded', y = "High_Salary_Estimate", data = ols_data2)
plt.show()

Traceback (most recent call last):
  File "C:\Users\marioreyes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Founded'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\marioreyes\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user